In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import models, layers
from fenpreprocessing import fen_to_array
from tensorflow.keras.callbacks import EarlyStopping
from data_generation import position_generator, fix_positions

import datetime
%load_ext tensorboard

In [2]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
# Setting paramaters on early stopping
earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=20,
                          verbose=1,
                          mode='min',
                          restore_best_weights=True)

log_dir = "logs/fit/baseline" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


In [4]:
# Memory management, likely not necessary, but used as a safety as per the documentation recommendations on using GPUS

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)


1 Physical GPUs, 1 Logical GPUs


In [5]:
small_train = pd.read_csv('fens/converted_train_partial.csv')
small_val = pd.read_csv('fens/converted_val_partial.csv')

In [12]:
train_gen = position_generator(small_train)
val_gen = position_generator(small_val)

In [8]:
baseline_model = models.Sequential()
baseline_model.add(layers.Conv2D(64, 4, padding='same', input_shape=(8,8,13), activation='relu'))
baseline_model.add(layers.MaxPooling2D(2))
baseline_model.add(layers.Conv2D(32, 2, padding='same', activation='relu'))
baseline_model.add(layers.Flatten())
baseline_model.add(layers.Dense(64, activation='relu'))
baseline_model.add(layers.Dense(1, activation='sigmoid'))
baseline_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=['acc'])
baseline_model.summary()

# Fitting the model
baseline_history = baseline_model.fit(x=train_gen,
                    # validation_data=val_gen,
                    steps_per_epoch=100,
                    epochs=30,
                    callbacks=[earlystop, tensorboard_callback]
                    )

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 8, 8, 64)          13376     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 32)          8224      
_________________________________________________________________
flatten_1 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                32832     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 54,497
Trainable params: 54,497
Non-trainable params: 0
__________________________________________________

In [10]:
baseline_model.save('NewModel.h5')

In [24]:
man_val = position_generator(small_val)

In [30]:
first = next(man_val)

In [31]:
first_pos, first_target = first

In [38]:
first_pos[0].shape

(8, 8, 13)

In [34]:
first_target

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.])

In [48]:
max(baseline_model.predict(first_pos))

array([0.08334204], dtype=float32)